In [1]:
from plotly.offline import init_notebook_mode
from sqlalchemy import create_engine

from query_flow.parsers.postgres_parser import PostgresParser
from query_flow.vizualizers.query_vizualizer import QueryVizualizer

In [2]:
%load_ext autoreload
%autoreload 2

init_notebook_mode(connected=True)

In [3]:
con_str = 'postgresql:///etrabelsi_thesis'


## Bad estimation for query 1

In [4]:
query_renderer = QueryVizualizer(parser=PostgresParser(explain_prefix="EXPLAIN(ANALYZE, COSTS, FORMAT JSON)"))

In [5]:
query ="""
SELECT titles.title_id
FROM titles
INNER JOIN crew ON crew.title_id = titles.title_id
INNER JOIN people ON people.person_id = crew.person_id
WHERE genres like '%Comedy%' 
  AND name in ('Owen Wilson', 'Adam Sandler', 'Jason Segel')
"""

In [6]:
query_renderer.parser.from_query(query, con_str)

{'Node Type': 'Gather',
 'Parallel Aware': False,
 'Startup Cost': 772761.07,
 'Total Cost': 935729.46,
 'Plan Rows': 11,
 'Plan Width': 10,
 'Actual Startup Time': 4326.166,
 'Actual Total Time': 5004.375,
 'Actual Rows': 558,
 'Actual Loops': 1,
 'Output': ['titles.title_id'],
 'Workers Planned': 2,
 'Workers Launched': 2,
 'Single Copy': False,
 'Shared Hit Blocks': 9298,
 'Shared Read Blocks': 592252,
 'Shared Dirtied Blocks': 0,
 'Shared Written Blocks': 0,
 'Local Hit Blocks': 0,
 'Local Read Blocks': 0,
 'Local Dirtied Blocks': 0,
 'Local Written Blocks': 0,
 'Temp Read Blocks': 0,
 'Temp Written Blocks': 0,
 'Plans': [{'Node Type': 'Hash Join',
   'Parent Relationship': 'Outer',
   'Parallel Aware': True,
   'Join Type': 'Inner',
   'Startup Cost': 771761.07,
   'Total Cost': 934728.36,
   'Plan Rows': 5,
   'Plan Width': 10,
   'Actual Startup Time': 4310.139,
   'Actual Total Time': 4992.795,
   'Actual Rows': 186,
   'Actual Loops': 3,
   'Output': ['titles.title_id'],
   'I

In [7]:
cardinality_df = query_renderer.get_cardinality_df(query, con_str=con_str)

In [8]:
query_renderer.vizualize(cardinality_df, title="Bad estimation for query 1", metrics=["actual_rows", "plan_rows"], open_=False)

In [ ]:
query_renderer.vizualize(cardinality_df, title="Bad estimation for query 1", metrics=["estimated_cost_pct", "actual_duration_pct"], open_=False)

## Bad estimation for query 2

In [ ]:
query = """
SELECT name
FROM titles
INNER JOIN crew ON crew.title_id = titles.title_id
INNER JOIN people ON people.person_id = crew.person_id
INNER JOIN ratings ON ratings.title_id = titles.title_id
WHERE died IS NULL
  AND genres LIKE '%Comedy%'
GROUP BY name
HAVING COUNT(*) > 10
ORDER BY AVG(rating) DESC
LIMIT 10
"""

In [ ]:
cardinality_df = query_renderer.get_cardinality_df(query, con_str=con_str)
query_renderer.vizualize(cardinality_df, title="Bad estimation for query 2", metrics=["actual_rows", "plan_rows"], open_=False)


In [ ]:
query_renderer.vizualize(cardinality_df, title="Bad estimation for query 2", metrics=["estimated_cost_pct", "actual_duration_pct"], open_=False)
